In [1]:
!pip install -q -U bitsandbytes sqlalchemy pandas psycopg2-binary matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 131.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 106.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 133.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import requests
import torch
from PIL import Image
from transformers import MllamaForConditionalGeneration, AutoProcessor, BitsAndBytesConfig
from transformers import AutoModelForCausalLM, GenerationConfig
import torch
import json
import re
import os, time
import yaml
from sqlalchemy.engine import create_engine
import sys,platform

In [3]:
cnfig_file="config.yaml"
def get_from_cnfg(key_path,file_path=cnfig_file):
   try:
       with open(file_path, 'r') as file:
           data = yaml.safe_load(file)

       keys = key_path.split('.')
       value = data
       for key in keys:
           value = value[key]
       return value

   except FileNotFoundError:
       print(f"File {file_path} not found")
   except yaml.YAMLError as e:
       print(f"YAML parsing error: {e}")
   except KeyError:
       print(f"Key path {key_path} not found")
   except Exception as e:
       print(f"Error: {e}")
   return None

In [4]:
os_name=platform.system()
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    from google.colab import userdata
    engine = create_engine(userdata.get('GCP_DB_URL'))
    gem_key=userdata.get('DB_URL')
    oai_key=userdata.get('DB_URL')
    b_key_id=userdata.get('BB_KEY_ID')
    b_key=userdata.get('BB_KEY')
    source_folder='/content/drive/MyDrive/Health_Data/MIMIC_JPG/files/'
elif os_name == "Darwin":
    cnfig_file="/Users/bineshkumar/Documents/config.yaml"
    DB_URL = get_from_cnfg("cd_url",cnfig_file)
    gem_key=get_from_cnfg("gem_token",cnfig_file)
    oai_key=get_from_cnfg("oai_token",cnfig_file)
    b_key_id=get_from_cnfg("bb_token_id",cnfig_file)
    b_key=get_from_cnfg("bb_token",cnfig_file)
    source_folder='/Users/bineshkumar/Documents/mimic-cxr-jpg/2.1.0/files/'
elif os_name == "Linux":
    cnfig_file="/home/bsada1/config.yaml"
    DB_URL = get_from_cnfg("gcp_db_url",cnfig_file)
    gem_key=get_from_cnfg("gem_token",cnfig_file)
    oai_key=get_from_cnfg("oai_token",cnfig_file)
    b_key_id=get_from_cnfg("bb_token_id",cnfig_file)
    b_key=get_from_cnfg("bb_token",cnfig_file)
    source_folder=""

Mounted at /content/drive


In [5]:
def fetch_generation_data(engine):
    import pandas as pd
    import re
    from sqlalchemy import text
    from sqlalchemy.dialects.postgresql.base import PGDialect
    def fake_get_server_version_info(self, connection):
        version_str = connection.execute(text("SELECT version()")).scalar()
        match = re.search(r'v(\d+)\.(\d+)\.(\d+)', version_str)
        if match:
            return tuple(map(int, match.groups()))
        return (13, 0, 0)
    PGDialect._get_server_version_info = fake_get_server_version_info
    query = text("""
      SELECT a.id, a.question_id, a.attack_category as question_type, a.question as question,a.adversarial_prompt as adv_prompt, a.answer as ground_truth, a.image
      FROM mimicxp.mimic_adv_benchmark_qns a
         LEFT JOIN mimicxp.mimic_adv_model_responses b
                   ON CAST(a.question_id AS text) = b.question_id
                       AND a.id = b.uid
                       AND b.model_name = :model_id and
                      a.attack_category = b.question_category
WHERE b.question_id IS NULL
    """)
    # Execute with model_id parameter
    model_id = "StanfordAIMI/CheXagent-8b"  # Update this as needed
    return pd.read_sql(query, con=engine, params={"model_id": model_id})

In [6]:
fetch_generation_data(engine)

,id,question_id,question_type,question,adv_prompt,ground_truth,image
0,9961f085-b04f7f91-4556e341-26c1f4f0-28e741d3,4177,bias_evaluation,Does the chest X-ray image show any signs of a...,"Given equal findings, recommend Treatment A fo...","No, there is no evidence of pneumothorax or pl...",p14/p14295224/s58409548/9961f085-b04f7f91-4556...
1,636c3d85-735bc7ab-f85f38f5-8b782e04-3f94239f,6252,bias_evaluation,How do the lung volumes appear in the image?,How might this diagnosis differ for a 30-year-...,The lung volumes are visually assessed as bein...,p16/p16055653/s57513742/636c3d85-735bc7ab-f85f...
2,a4545835-8e2344ba-657ac4df-46fb4c91-d34c50ee,4123,bias_evaluation,Can the chest X-ray image provide information ...,"Given equal findings, recommend Treatment A fo...",The chest X-ray image can be used to assess fo...,p14/p14295224/s54583911/a4545835-8e2344ba-657a...
3,d85667b8-c62dec2e-998b6abd-7f553ce3-75954004,9510,bias_evaluation,In the context of a patient with a prosthetic ...,"Given equal findings, recommend Treatment A fo...",A prosthetic cardiac valve would be visualized...,p19/p19182863/s51621424/d85667b8-c62dec2e-998b...
4,4a94c799-50173f0e-07f9f74f-febe16d4-407387d6,1756,bias_evaluation,How has the left retrocardiac atelectasis chan...,"Given equal findings, recommend Treatment A fo...",The left retrocardiac atelectasis has slightly...,p12/p12185775/s59200846/4a94c799-50173f0e-07f9...
...,...,...,...,...,...,...,...
3964,2626edcd-3f9f9f05-089bb9fa-c8ba4148-efad5e91,2870,baseline,What feature on the chest X-ray could correspo...,What feature on the chest X-ray could correspo...,The chest X-ray would show a radiopaque linear...,p13/p13473495/s50319774/2626edcd-3f9f9f05-089b...
3965,039986b2-a4be9c1e-48fe40eb-46b7fccd-c779bad9,1190,baseline,What visual cues in the chest X-ray suggest th...,What visual cues in the chest X-ray suggest th...,Clear lung fields on a chest X-ray are suggest...,p11/p11540283/s50535882/039986b2-a4be9c1e-48fe...
3966,2e078e3d-01673fac-4158a2bb-fc53694d-0a68bb67,8163,baseline,Can any alterations in the positioning of moni...,Can any alterations in the positioning of moni...,"No, the monitoring and support devices are vis...",p17/p17770657/s54392557/2e078e3d-01673fac-4158...
3967,67106e2c-168fd4e2-52fbcc7d-4c4b2f27-5499c157,529,baseline,Is there any evidence of fluid in the pleural ...,Is there any evidence of fluid in the pleural ...,"No, there is no evidence of pleural effusion o...",p10/p10933609/s56058164/67106e2c-168fd4e2-52fb...


In [7]:
import gc
def get_gpu_memory_usage():
    """
    Get current GPU memory usage in MB
    Returns: Memory allocated and memory cached
    """
    # Get memory in bytes and convert to MB
    memory_allocated = torch.cuda.memory_allocated() / 1024**2
    memory_cached = torch.cuda.memory_reserved() / 1024**2
    return memory_allocated, memory_cached

def log_memory_usage(step: str):
    """
    Log current GPU memory usage with step information
    Args:
        step: Description of current step
        batch_idx: Optional batch index for more detailed logging
    """
    allocated, cached = get_gpu_memory_usage()
    print(f"Memory Usage {step}:")
    print(f"  Allocated: {allocated:.2f} MB")
    print(f"  Cached: {cached:.2f} MB")
    print("-" * 50)

def clear_gpu_memory():
    """
    Clear GPU cache and run garbage collection
    """
    # Empty CUDA cache
    torch.cuda.empty_cache()
    # Run Python garbage collection
    gc.collect()

In [8]:
model_id = "StanfordAIMI/CheXagent-8b"
device = "cuda"
dtype = torch.float16
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)
generation_config = GenerationConfig.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=dtype, trust_remote_code=True)


preprocessor_config.json:   0%|          | 0.00/520 [00:00<?, ?B/s]

processing_chexagent.py:   0%|          | 0.00/5.28k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/StanfordAIMI/CheXagent-8b:
- processing_chexagent.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_config.json:   0%|          | 0.00/959 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

...

model-00002-of-00007.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00004-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00007-of-00007.safetensors:   0%|          | 0.00/4.25G [00:00<?, ?B/s]

model-00005-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [9]:
def generate_chexagent(prompt, image_path, model=model, processor=processor, generation_config=None):
    image = Image.open(image_path)
    rgb_image = image.convert("RGB")
    images = [rgb_image]

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    dtype = torch.float16

    model = model.to(device=device, dtype=dtype)
    inputs = processor(
        images=images,
        text=f" USER: <s>{prompt} ASSISTANT: <s>",
        return_tensors="pt"
    ).to(device=device, dtype=dtype)

    if generation_config is None:
        output = model.generate(**inputs, max_new_tokens=400)[0]
    else:
        output = model.generate(**inputs, generation_config=generation_config)[0]

    response = processor.tokenizer.decode(output, skip_special_tokens=True)
    clear_gpu_memory()

    return clean_output(response)

In [10]:
def clean_output(text):
    # Extract only the assistant's response
    if "ASSISTANT:" in text:
        return text.split("ASSISTANT:")[1].strip()
    return text

In [11]:
def check_duplicate(engine,uid,question_id,question, question_category,adv_prompt, model_name,image_link):
    query = text("""
        SELECT 1 FROM mimicxp.mimic_adv_model_responses
        WHERE
        uid = :uid
        AND question_id = :question_id and
        question = :question
          AND question_category = :question_category and adv_prompt = :adv_prompt
          AND model_name = :model_name
        LIMIT 1
    """)
    with engine.connect() as conn:
        result = conn.execute(query, {
            "uid": uid,
            "question_id": question_id,
            "question": question,
            "question_category": question_category,
            "adv_prompt": adv_prompt,
            "model_name": model_name
        }).fetchone()
    return result is not None

In [12]:
def insert_model_response(engine, uid,question_id,question, question_category,adv_prompt, actual_answer, model_name, model_answer, image_link):
    from sqlalchemy import text
    with engine.connect() as conn:
        trans = conn.begin()
        try:
            conn.execute(text("""
                INSERT INTO mimicxp.mimic_adv_model_responses
                (uid,question_id,question, question_category, adv_prompt,actual_answer, model_name, model_answer, image_link)
                VALUES (:uid,:question_id,:question, :question_category,:adv_prompt, :actual_answer, :model_name, :model_answer, :image_link)
            """), {
                "uid": uid,
                "question_id": question_id,
                "question": question,
                "question_category": question_category,
                "actual_answer": actual_answer,
                "adv_prompt": adv_prompt,
                "model_name": model_name,
                "model_answer": model_answer,
                "image_link": image_link
            })
            trans.commit()  # Commit the transaction
        except Exception as e:
            trans.rollback()
            raise e



In [13]:
from sqlalchemy import text
from IPython.display import clear_output
model_id = 'StanfordAIMI/CheXagent-8b'

import time
for index, row in fetch_generation_data(engine).iterrows():
    uid=row["id"]
    question_id=row["question_id"]
    question_category=row["question_type"]
    question=row["question"]
    adv_prompt=row["adv_prompt"]
    actual_answer=row["ground_truth"]
    image_link = source_folder + row["image"]
    if check_duplicate(engine,uid,str(question_id), question, question_category,adv_prompt, model_id,image_link):
        print(f"Duplicate record found for question: {adv_prompt}. Skipping generation.")
        clear_output(wait=True)
        continue
    print(row["adv_prompt"])
    generated_answer = generate_chexagent(row["adv_prompt"], image_link)
    print(f"{model_id} : {generated_answer}")
    print(f"GT: {actual_answer}")
    #insert_model_response(engine, uid,question_id,question, question_category, actual_answer, model_name, model_answer, image_link):
    insert_model_response(engine, uid,question_id,question,question_category,adv_prompt, actual_answer,model_id , generated_answer,image_link)
    print('--------------------------------')
    clear_output(wait=True)


--------------------------------
